# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("../output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,edinburgh of the seven seas,-37.0676,-12.3116,13.13,83,68,10.08,SH,1698187004
1,1,jamestown,42.0970,-79.2353,21.80,30,0,3.60,US,1698187004
2,2,port mathurin,-19.6833,63.4167,24.53,80,17,6.83,MU,1698187004
3,3,puerto natales,-51.7236,-72.4875,6.19,74,98,6.17,CL,1698187004
4,4,bilibino,68.0546,166.4372,-14.94,96,30,1.66,RU,1698187004


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
%%capture --no-display


map_plot_4 = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500,
    size = "Humidity",
    scale = 1,
    color = "City",
    alpha=0.5
)

# Display the map plot
map_plot_4


:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [4]:
# Narrow down cities that fit criteria and drop any results with null values
# YOUR CODE HERE Narrow down the city_data_df DataFrame to find your ideal weather condition. For example:

ideal_vacation_df = city_data_df [(city_data_df["Max Temp"]>21) & (city_data_df["Max Temp"]<27) & (city_data_df["Wind Speed"]<4.5)]
ideal_vacation_df = ideal_vacation_df.dropna(how='any')
ideal_vacation_df

# Drop any rows with null values
ideal_vacation_df= ideal_vacation_df.dropna(how='any')

# Display sample data
ideal_vacation_df.head()


,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
1,1,jamestown,42.0970,-79.2353,21.80,30,0,3.60,US,1698187004
5,5,bengkulu,-3.8004,102.2655,23.08,93,96,1.05,ID,1698187004
12,12,chiredzi,-21.0500,31.6667,26.37,45,0,1.46,ZW,1698187005
27,27,adamstown,-25.0660,-130.1015,22.57,81,1,4.27,PN,1698187008
30,30,pacific grove,36.6177,-121.9166,23.47,82,40,3.60,US,1698187008


### Step 3: Create a new DataFrame called `hotel_df`.

In [5]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity


columns_to_copy=["City", "Country", "Lat", "Lng", "Humidity"]
hotel_df= ideal_vacation_df[columns_to_copy].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API

hotel_df['Hotel Name']= ""

# displaying the sample of Data Frame
hotel_df.head()

,City,Country,Lat,Lng,Humidity,Hotel Name
1,jamestown,US,42.0970,-79.2353,30,
5,bengkulu,ID,-3.8004,102.2655,93,
12,chiredzi,ZW,-21.0500,31.6667,45,
27,adamstown,PN,-25.0660,-130.1015,81,
30,pacific grove,US,36.6177,-121.9166,82,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [6]:
# Set parameters to search for a hotel
radius = 10000

params = {
    "categories":"accommodation.hotel",
    "limit":20,
    "filter":{},
    "bias": {},
    "apiKey":geoapify_key 
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    
    # get latitude, longitude from the DataFrame
    longitude=row['Lng']
    latitude= row['Lat']
    
 
     # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] =f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places?"
    #https://api.geoapify.com/v2/places?categories=healthcare&filter=rect:-77.03921757581298,38.90660286449031,-77.01762874146101,38.88987145915573&limit=100&apiKey=YOUR_API_KEY

    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params = params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
jamestown - nearest hotel: DoubleTree Jamestown
bengkulu - nearest hotel: Dena Hotel
chiredzi - nearest hotel: Nesbitt Arms Hotel
adamstown - nearest hotel: No hotel found
pacific grove - nearest hotel: Pacific Grove Inn
puerto leguizamo - nearest hotel: Hotel Girasoles
flying fish cove - nearest hotel: Christmas Island Lodge
fort bragg - nearest hotel: Airborne Inn Lodging
yopal - nearest hotel: Hotel Sophia Real
tazacorte - nearest hotel: App Leyma
solwezi - nearest hotel: Lodge 24
avarua - nearest hotel: Paradise Inn
anloga - nearest hotel: Hotel de White House
aras-asan - nearest hotel: Cagwait LGU Beach Resort
taradale - nearest hotel: Ali Almajidi
porto franco - nearest hotel: Hotel Belém-Brasilia
chemin grenier - nearest hotel: Hotel Shanti Maurice
pinotepa nacional - nearest hotel: No hotel found
tolanaro - nearest hotel: Hôtel Mahavokey
kerikeri - nearest hotel: Avalon Resort
siracusa - nearest hotel: Hotel Panorama
puerto rico - nearest hotel: Suizo
geor

,City,Country,Lat,Lng,Humidity,Hotel Name
1,jamestown,US,42.0970,-79.2353,30,DoubleTree Jamestown
5,bengkulu,ID,-3.8004,102.2655,93,Dena Hotel
12,chiredzi,ZW,-21.0500,31.6667,45,Nesbitt Arms Hotel
27,adamstown,PN,-25.0660,-130.1015,81,No hotel found
30,pacific grove,US,36.6177,-121.9166,82,Pacific Grove Inn
...,...,...,...,...,...,...
575,upata,VE,8.0086,-62.3989,96,Hotel Garibardi
579,paka,MY,4.6374,103.4368,94,Paka Inn
580,harper,LR,4.3750,-7.7169,91,No hotel found
583,ulladulla,AU,-35.3500,150.4667,68,Hotel Marlin


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [7]:
%%capture --no-display

# Configure the map plot
map_plot_2 = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 600,
    size = "Humidity",
    scale = 1,
    color = "City",
    alpha=0.5,
    hover_cols=['Country', 'Hotel Name']
)

    
# Display the map
map_plot_2

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Country,Hotel Name)